# SkillFactory
## Hakaton Kaggle

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

from datetime import datetime
from pprint import pprint

from sklearn.cluster import KMeans
from tqdm import tqdm_notebook

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier

#from nltk.stem import SnowballStemmer
import math


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)
#snowball_stemmer = SnowballStemmer( "russian" )

#from pymystem3 import Mystem

#m = Mystem()

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
params = {'20': ['м²'], '100':{'give':['Сдам', 'Сдаю','сдам','сдаю','Сдаётся','сдаётся', 'Снять','снять','Здамs'\
                                      ,'Сдается','сдается','Аренда','аренда'],\
                               'sell':['Продам','Продаю','Квартира','Продается','Гараж','Гораж','Коттедж','Дача',\
                                      'продам','продаю','квартира','продается','гараж','гораж','коттедж','дача'\
                                      ,'Участок','участок','Дом','дом','Дома','дома']}}

# Грузим целый трейн

In [2]:
%%time

df_train = pd.read_pickle('hackathon-sf-ml-11/train_hack.pckl')
df_train = df_train.set_index('id')

CPU times: user 2min 14s, sys: 2min 31s, total: 4min 45s
Wall time: 5min 12s


# грузим целый тест

In [4]:
%%time

df_test = pd.read_pickle('hackathon-sf-ml-11/test_hack.pckl')
df_test = df_test.set_index('id')

CPU times: user 1min 37s, sys: 4min 4s, total: 5min 41s
Wall time: 6min 40s


# Проверяем, что все нормально загрузилось

In [5]:
df_test[:1]

,can_buy,can_promote,category,contacts_visible,date_created,delivery_available,description,fields,images,location,mortgage_available,name,payment_available,subcategory,subway
id,,,,,,,,,,,,,,,
285ea2e9935ccdeb8378c6a5,False,False,6,True,1517061944,True,"Стремянка трехсекционная,3×4","[{'field': {'name': 'Ремонт и строительство', ...","[{'id': '5a6c864a074b3e7d207f3522', 'num': 1, ...","{'latitude': 54.749969, 'longitude': 55.993774}",False,Стремянка,True,603,NaN


# Дропаем ненужные столбцы

In [6]:
df_train = df_train.drop(['description', 'fields',], axis=1)

In [7]:
df_test = df_test.drop(['description', 'fields',], axis=1)

# Чистим дату

In [8]:
df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'score'



# Concat
concat_df = df_test.append(df_train)



In [9]:
concat_df.head()

,can_buy,can_promote,category,contacts_visible,date_created,delivery_available,images,location,mortgage_available,name,payment_available,price,sample,subcategory,subway
id,,,,,,,,,,,,,,,
285ea2e9935ccdeb8378c6a5,False,False,6,True,1517061944,True,"[{'id': '5a6c864a074b3e7d207f3522', 'num': 1, ...","{'latitude': 54.749969, 'longitude': 55.993774}",False,Стремянка,True,NaN,score,603,NaN
adfb73820bbb831257df6e95,False,False,2,True,1508310389,False,"[{'id': '59e6fc49f202633e173ad413', 'num': 1, ...","{'latitude': 59.737736, 'longitude': 30.078828}",False,Плита электрическая,True,NaN,score,203,NaN
783025601c36202f633fc6a5,False,False,1,True,1517089590,False,"[{'id': '5a6cf2b65eaa9e4973249b62', 'num': 1, ...","{'latitude': 55.989156, 'longitude': 37.145419}",False,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,False,NaN,score,116,NaN
2f0cd2d2e15dc90afd847f95,False,False,10,True,1509378271,False,"[{'id': '59f748bca09cd5081f40d786', 'num': 1, ...","{'latitude': 55.805178, 'longitude': 37.771612}",False,Batman Archam Knight,True,NaN,score,1009,NaN
5c23a37902855a20172845a5,False,False,11,True,1515487857,False,"[{'id': '5a548186c6ab9e73a2056162', 'num': 1, ...","{'latitude': 59.843029, 'longitude': 30.246231}",False,Ледобур для зимней рыбалки,False,NaN,score,1104,NaN


In [10]:
concat_df.loc[concat_df['price'].isnull(), 'expensive'] = 0
concat_df['expensive'] = np.where(concat_df['price'] > 100000000, 1, 0)

In [ ]:
for idx, row in concat_df[concat_df['category'] == 20].iterrows():
    
    metersTrue = False
    meters_number = 0
    meters_sum=0
    meters_average = 0
        
    for x in concat_df.loc[idx,'name'].split(' '):
        
        try:
            k = int(float(x))
            if k>9:
                meters_number +=1
                meters_sum += k
        except ValueError:
            p = 10

        if x in params['20']:
            metersTrue = True
    
    
    if metersTrue and meters_sum>0:
        meters_average = meters_sum // meters_number
        if meters_average < 30:
            concat_df.loc[idx,'name'] = 'small_meters'
        elif meters_average <= 50:
            concat_df.loc[idx,'name'] = 'medium_meters' 
        elif meters_average <= 80:
            concat_df.loc[idx,'name'] = 'big_meters'
        else:
            concat_df.loc[idx,'name'] = 'very_big_meters'

In [ ]:
for idx, row in concat_df[concat_df['category'] == 100].iterrows():
    
    for x in concat_df.loc[idx,'name'].split(' '):
        
        if x in params['100']['give']:
            concat_df.loc[idx,'name'] = 'give'
        
        if x in params['100']['sell']:
            concat_df.loc[idx,'name'] = 'sell'

In [ ]:
# дропаем can_buy, can_promote - т.к. всегда false или true, дропаем delivery_available, subway т.к. очень мало True

concat_df = concat_df.drop(['can_buy', 'can_promote', 'contacts_visible', 'mortgage_available',\
                          'delivery_available', 'subway'], axis=1)

#Dates...
concat_df['date_created'] = pd.to_datetime(concat_df['date_created'], unit='s')
concat_df['year'] = concat_df.date_created.dt.year
concat_df['month'] = concat_df.date_created.dt.month
concat_df['dow'] = concat_df.date_created.dt.dayofweek
concat_df.drop(['date_created'], axis=1, inplace=True)

# images_num делаем переменную - количество картинок
concat_df['images_num'] = concat_df['images'].str.len()
concat_df.drop(['images'], axis=1, inplace=True)

# превращаем location в 2 столбца

concat_df['latitude'] = concat_df['location'].apply(lambda x: x['latitude'])
concat_df['longitude'] = concat_df['location'].apply(lambda x: x['longitude'])
concat_df.drop(['location'], axis=1, inplace=True)




In [ ]:
loc_x = []
loc_y = []
for _ in tqdm_notebook( range(len(concat_df))) :
   loc_x.append(concat_df.iloc[_].latitude)
   loc_y.append(concat_df.iloc[_].longitude)

tmp_df = pd.DataFrame([loc_x, loc_y]).T

kmc = KMeans(n_clusters=85)
kmc.fit(tmp_df)
concat_df['cluster'] = kmc.labels_

In [ ]:
concat_df.drop(['latitude'], axis=1, inplace=True)
concat_df.drop(['longitude'], axis=1, inplace=True)

In [ ]:
# превращаем самые популярные категории из name в dummy

threshold = 500
name_popular_list=['sell','give','small_meters','medium_meters','big_meters','very_big_meters']

counts = concat_df.name.value_counts()
name_popular_list = counts[counts > threshold].index

concat_df['name'] = concat_df['name'].apply(lambda x: x if x in name_popular_list else 'other_name')

concat_df = pd.get_dummies(concat_df, columns=['name']) 


In [ ]:
concat_df = pd.get_dummies(concat_df, columns=['category', 'cluster']) 
# Split your data

# Drop your labels

df_train = concat_df.query('sample == "train"').drop(['sample'], axis=1)
df_test = concat_df.query('sample == "score"').drop(['sample'], axis=1)

In [ ]:
y_train.head()

In [ ]:
y_train = df_train['price']/100

X_train = df_train.drop(['price'], axis=1)

In [ ]:
X_train

# Делим на трейн и тест

In [ ]:
df_test = df_test.drop(['price'], axis=1)

In [ ]:
dtc = DecisionTreeRegressor()
dtc.fit(X_train, y_train)
predict_dtc = dtc.predict(df_test)

In [ ]:
y_hat = predict_dtc
df_test.loc[:, 'price'] = y_hat
df_submission = df_test['price']
df_submission.to_csv('submission_team3_on_v2.csv')
df_submission.head()

In [34]:
len(name_popular_list)

114